In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('darkgrid')

import nltk
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import re,string,unicodedata
from sklearn.feature_extraction.text import TfidfTransformer

from keras.preprocessing import text,sequence
from nltk.tokenize.toktok import ToktokTokenizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
from nltk import pos_tag
from nltk.util import ngrams
from nltk.corpus import wordnet
import keras
from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding
from keras.callbacks import ReduceLROnPlateau
import tensorflow as tf
import os
import numpy as np
import pandas as pd
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import pickle



In [8]:

nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/tristan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/tristan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [12]:
data = pd.DataFrame()
datadir = 'data/twittertf'
realorfake = {"True":1,"Fake":0}

for dir in os.listdir(datadir):
    temp_data = pd.read_csv(datadir + "/" +  dir)
    temp_data = temp_data.assign(real = realorfake.get(dir.split(".")[0]) )
    data = data.append(temp_data)

In [13]:
data.sample(5)

,title,text,subject,date,real
19728,GUESS WHO’S BEHIND Sickening Ad Showing Boy Be...,This ad is disgusting in so many ways. The vid...,left-news,"Oct 27, 2016",0
8206,Clinton regrets calling 'half' of Trump suppor...,NEW YORK (Reuters) - Democratic presidential c...,politicsNews,"September 10, 2016",1
7116,Should I stay or should I go? U.S. civil serva...,WASHINGTON (Reuters) - Donald Trump’s surprise...,politicsNews,"November 22, 2016",1
17426,THEIR EVIL KNOWS NO BOUNDS: TALIBAN EXECUTES 5...,The Taliban or ISIS? Does it really matter? Ra...,Government News,"Apr 13, 2015",0
6488,Trump calls storm over Russia hacking 'politic...,WASHINGTON (Reuters) - President-elect Donald ...,politicsNews,"January 6, 2017",1


In [14]:
stop_words=set(stopwords.words('english'))
punctuation=list(string.punctuation)
stop_words.update(punctuation)

In [15]:
def string_html(text):
    soup=BeautifulSoup(text,"html.parser")
    return soup.get_text()

def remove_square_brackets(text):
    return re.sub('\[[^]]*\]','',text)

def remove_URL(text):
    return re.sub(r'http\S+','',text)

def remove_stopwords(text):
    final_text=[]
    for i in text.split():
        if i.strip().lower() not in stop_words:
            final_text.append(i.strip())
    return " ".join(final_text)

def remove_HashOrAT(text):
    return re.sub('@|#','',text)

def remove_puncuation(text):
    return text.translate(str.maketrans('','',string.punctuation))

def remove_uppercase(text):
    return text.lower()
    

def clean_text_data(text):
    text=string_html(text)
    text=remove_square_brackets(text)
    text=remove_URL(text)
    text=remove_HashOrAT(text)
    text=remove_puncuation(text)
    text=remove_uppercase(text)
    return text

In [ ]:

#data['text']=data['text'].apply(clean_text_data)

In [16]:
X_train,X_test,y_train,y_test=train_test_split(data.text,data.real,random_state=0)

In [17]:
text_clf = Pipeline(
    [('vec', CountVectorizer(preprocessor=clean_text_data, ngram_range=(1,3))),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
    ])


In [18]:
text_clf = text_clf.fit(X_train, y_train)

/home/tristan/anaconda3/envs/mir/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://www.youtube.com/watch?v=PjeOoJyPNCk" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/tristan/anaconda3/envs/mir/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://100percentfedup.com/served-roy-moore-vietnamletter-veteran-sets-record-straight-honorable-decent-respectable-patriotic-commander-soldier/" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/tristan/anaconda3/envs/mir/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://www.youtube.com/watch?v=Ws5ojb0PCCo" looks like a 

In [19]:
predicted = text_clf.predict(X_test)
np.mean(predicted == y_test)

/home/tristan/anaconda3/envs/mir/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://www.youtube.com/watch?v=ISm-p8e-D7I" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/tristan/anaconda3/envs/mir/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://www.youtube.com/watch?v=9LNyx_DWzzA" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  warnings.warn(
/home/tristan/anaconda3/envs/mir/lib/python3.8/site-packages/bs4/__init__.py:431: MarkupResemblesLocatorWarning: "https://youtu.be/RTuxvWjH3a4" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document b

0.9652561247216036

In [20]:
text_clf.predict(["Donald Trump is President of the United States"])

array([0])

In [21]:
with open("twitterTFModel.pkl",'wb') as file:
    pickle.dump(text_clf, file)
    

In [22]:
class model1_tweets:
    def __init__(self):
        self.model = pickle.load(open("twitterTFModel.pkl",'rb'))
        self.key = realorfake = {"True":1,"False":0}
        
    
    def preditWithText(self, text):
        return self.model.predict([text])[0]

    def return_str(self, text):
        if self.preditWithText(text) == 0:
            return False
        else:
            return True
        

In [23]:
newText = model1_tweets()
newText.preditWithText("Donald Trump is President")

0